## Normalizatoin

#### L1 정규화 (Lasso)

In [12]:
import torch, torch.nn as nn, torch.optim as optim

l1_lambda = 0.001 # L1 정규화 강도

model = nn.Linear(in_features=10, out_features=1) # 간단한 모델 정의

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

inputs = torch.randn(32, 10)  # 임의 입력데이터
labels = torch.randn(32, 1)   # 임의 타겟데이터

optimizer.zero_grad()  # 1 단계 학습
outputs = model(inputs)
base_loss = criterion(outputs, labels) # 기본 손실

l1_norm = sum(p.abs().sum() for p in model.parameters())
total_loss = base_loss + l1_lambda * l1_norm # L1 페널티가 추가된 최종 손실

total_loss.backward()
optimizer.step()

print(f"Base Loss: {base_loss.item()}")
print(f"L1 Regularized Total Loss: {total_loss.item()}") # L1 항 때문에 기본 손실보다 큼

Base Loss: 1.4051369428634644
L1 Regularized Total Loss: 1.4070558547973633


#### L2 정규화 (Weight Decay)

In [13]:
import torch, torch.nn as nn, torch.optim as optim

model = nn.Linear(in_features=10, out_features=1) # 간단한 모델 정의

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, 
                       weight_decay=1e-5) # 1e-5가 L2 정규화 강도

inputs = torch.randn(32, 10)
labels = torch.randn(32, 1)

optimizer.zero_grad()  # 1 단계 학습 (L1과 달리 손실 함수 수정 불필요)
outputs = model(inputs)
loss = criterion(outputs, labels)
loss.backward()
optimizer.step()

print(f"L2 Regularized Loss: {loss.item()}")

L2 Regularized Loss: 1.47163724899292


#### 드롭아웃 (Dropout)

In [21]:
import torch, torch.nn as nn, torch.optim as optim

model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Dropout(p=0.5),  # p=0.5는 훈련 중 50%의 뉴런을 비활성화한다는 의미
    nn.Linear(128, 10)
)

inputs = torch.randn(32, 784)

model.train()
train_output = model(inputs)

with torch.no_grad(): # model의 첫 Linear, ReLU, Dropout까지 실행
	x = model[0](inputs)
	x = model[1](x)
	dropout_output = model[2](x)
print(dropout_output)

tensor([[1.4145, 0.2331, 0.0000,  ..., 0.0000, 0.0000, 0.4686],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2751,  ..., 0.0000, 0.7619, 0.0000],
        ...,
        [0.0000, 0.6044, 0.0000,  ..., 0.0000, 0.0000, 0.0046],
        [0.9105, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7438],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


#### 배치 정규화 (Batch Normalization)

In [ ]:
import torch, torch.nn as nn, torch.optim as optim

model_mlp = nn.Sequential(
    nn.Linear(20, 128),
    nn.BatchNorm1d(128) # Linear 출력 128개를 정규화
)
model_mlp.train() # BatchNorm은 train 모드에서 실행해야 함

inputs = torch.randn(32, 20) 

linear_output = model_mlp[0](inputs) 
print("Before BatchNorm")
print(f"Mean: {linear_output.mean().item():.4f}") # 128개 특성(채널) 전체의 평균/표준편차
print(f"Std:  {linear_output.std().item():.4f}")

bn_output = model_mlp(inputs) # BatchNorm까지 통과
print("\nAfter BatchNorm") # 128개 특성 전체의 평균은 0에, 표준편차는 1에 가깝게 보정됨
print(f"Mean: {bn_output.mean().item():.4f}") # 0에 매우 가까움
print(f"Std:  {bn_output.std().item():.4f}")  # 1에 매우 가까움

--- Before BatchNorm ---
Mean: -0.0003
Std:  0.5889

--- After BatchNorm ---
Mean: -0.0000
Std:  1.0001


#### 레이어 정규화 (Layer Normalization)

In [ ]:
import torch, torch.nn as nn, torch.optim as optim

# (N, SeqLen, Features) 형태의 3D 텐서를 가정
batch_size = 4
seq_len = 5
embedding_dim = 8 # 정규화할 특성의 차원

input_tensor = torch.rand(batch_size, seq_len, embedding_dim) * 5 # 입력

layer_norm = nn.LayerNorm(normalized_shape=embedding_dim) 
output_tensor = layer_norm(input_tensor) #  레이어 정규화 적용

print("Before LayerNorm (Sample 0)")
print(f"Mean: {input_tensor[0].mean().item():.4f}") # 0번 샘플의 평균/표준편차
print(f"Std:  {input_tensor[0].std().item():.4f}")

print("\nAfter LayerNorm (Sample 0)") # 0번 샘플이 평균 0, 표준편차 1로 정규화됨
print(f"Mean: {output_tensor[0].mean().item():.4f}") # 0에 매우 가까움
print(f"Std:  {output_tensor[0].std().item():.4f}")  # 1에 매우 가까움

--- Before LayerNorm (Sample 0) ---
Mean: 2.7334
Std:  1.3637

--- After LayerNorm (Sample 0) ---
Mean: 0.0000
Std:  1.0127


#### 그룹 정규화 (Group Normalization)

In [ ]:
import torch, torch.nn as nn, torch.optim as optim

# (N, C, H, W) 형태의 4D 텐서를 가정 (CNN)
batch_size = 4
num_channels = 6
H, W = 5, 5

input_tensor = torch.rand(batch_size, num_channels, H, W) * 5 # 입력

num_groups = 3 # 6개 채널을 3개 그룹으로 나눔 (그룹당 2개 채널)
group_norm = nn.GroupNorm(num_groups=num_groups, 
                          num_channels=num_channels) # GroupNorm 레이어 정의

output_tensor = group_norm(input_tensor) # 그룹 정규화 적용

print("Before GroupNorm (Sample 0, Group 0)")
group0_input = input_tensor[0, :2, :, :] # 0 ~ 1번 채널
print(f"Mean: {group0_input.mean().item():.4f}")
print(f"Std:  {group0_input.std().item():.4f}")

print("\nAfter GroupNorm (Sample 0, Group 0)")
group0_output = output_tensor[0, :2, :, :] # 0번 샘플의 0번 그룹이 평균 0, 표준편차 1로 정규화됨
print(f"Mean: {group0_output.mean().item():.4f}") # 0에 매우 가까움
print(f"Std:  {group0_output.std().item():.4f}")  # 1에 매우 가까움

--- Before GroupNorm (Sample 0, Group 0) ---
Mean: 2.4736
Std:  1.4974

--- After GroupNorm (Sample 0, Group 0) ---
Mean: 0.0000
Std:  1.0102
